In [2]:
!pip install lxml

In [3]:
import pandas as pd

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url="https://hoopshype.com/salaries/players/"

In [5]:
r = requests.get(url,timeout=2.5)
r_html = r.text

soup = BeautifulSoup(r_html, 'html.parser')

In [6]:
links = { a.text.strip(): a['href'] for a in soup.find("div", {"class": "salaries-team-selector-top"}).find_all("a", href=True, text=True) }

In [7]:
salaries = { season: pd.read_html(link)[0] for season, link in links.items() }

In [8]:
df_salaries = [ df.drop(columns='Unnamed: 0').set_index('Player') for df in salaries.values() ]

In [151]:
salary = df_salaries[0].join(df_salaries[1:], how='outer')

In [156]:
salary = salary.replace({'\$':'', ',':''}, regex=True)

In [157]:
salary.columns = salary.columns.str.replace('/', '-').str.replace('(*)', '_deflacted', regex=False)

In [158]:
nominal_cols = [i for i in salary.columns if 'deflacted' not in i]
real_cols = [i for i in salary.columns if 'deflacted' in i]

In [159]:
final_salary = pd.DataFrame()
for row in salary.iterrows():
    name = row[0]
    nominal = row[1][nominal_cols].rename('nominal').to_frame()
    real = row[1][real_cols].rename(lambda x: x.replace('_deflacted', '')).rename('real').to_frame()
    player = nominal.join(real).sort_index().reset_index().rename(columns={'index': 'season'}).dropna(how='all', subset=['nominal', 'real'])
    player.insert(0, 'player', name)
    final_salary = final_salary.append(player)

In [179]:
final_salary.head(20)

,player,season,nominal,real
0,A.C. Green,1990-91,1750000,3473015
1,A.C. Green,1991-92,1750000,3317240
2,A.C. Green,1992-93,1750000,3217865
3,A.C. Green,1993-94,1885000,3365286
4,A.C. Green,1994-95,6472600,11274438
5,A.C. Green,1995-96,6473000,10942426
6,A.C. Green,1996-97,4851000,7980684
7,A.C. Green,1997-98,5095088,8194001
8,A.C. Green,1998-99,5125088,8105719
9,A.C. Green,1999-00,1700000,2636912


In [161]:
final_salary.to_csv('../data/salary.csv', index=False)